In [12]:
from rec_room.recommenders import BaseRS
from rec_room.datasets import load_us_colleges_dataset
from rec_room import ROOT_DIR

In [3]:
from os.path import join
from typing import Dict

import joblib

In [4]:
RS_PATH = join(ROOT_DIR, 'recommenders')

In [55]:
class USCollegesRS(BaseRS):
    
    META = dict(
        dataset = 'us_colleges.csv',
        model = 'us_colleges_model.sav',
        labels = 'us_colleges_labels.sav',
        path = 'us_colleges_rs'
    )
    
    def __init__(self, rs_args:Dict[str, str]=None, rs_dir:str=None) -> None:
        """
        Instantiate the US College Recommender System.

        Parameters
        ----------
        rs_args : Dict
            User specified arguments for preferences or constraints
            
        rs_dir : Str
            Directory to where the RS is located.
        """
        if rs_dir is None:
            rs_dir = join(RS_PATH, self.META['path'])            
            
        super(USCollegesRS, self).__init__(
            args=rs_args,
            path=rs_dir            
        )
        
        self._model = joblib.load(join(self.path, self.META['model']))
        self._labels = joblib.load(join(self.path, self.META['labels']))

        
    @property
    def dataset(self) -> 'pd.DataFrame':
        return load_us_colleges_dataset()
    
    @property
    def model(self) -> 'sklearn.neighbors.NearestNeighbor':
        """
        NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                         metric_params=None, n_jobs=None, n_neighbors=161, 
                         p=2, radius=1.0)
        """
        return self._model
        
    @property
    def labels(self) -> 'pd.DataFrame':
        return self._labels
    
    def recommend(self, args=None) -> Dict[str, str]:
        """
        Make a Recommendation for US Colleges.

        This function calls the pre-trained `self.model` and `self.labels`
        to make the best prediction, or recommendation, 
        for the provided instance arguments.

        Parameters
        ----------
        args : Dict
            User specified arguments for preferences or constraints

        Returns
        -------
        Dict
            The recommendation results
        """
        if args is None:
            args = self.args
        
        assert args is not None
        
        metrics = args.get('metrics', None)
        choices = args.get('choices', [])
        top = args.get('top', len(choices))
        
        assert metrics is not None
        
        inst = [metrics['gre'], metrics['gpa'], metrics['rank']]
        
        preds = self.model.kneighbors([inst], return_distance=False)[0]        
        recs = self.dataset.iloc[preds]              
        recs = recs[recs['name'].isin(choices)].reset_index(drop=True)[:top]
                
        return recs.to_dict('records')
    
    def render(self) -> 'HTML':
        pass    
            
    def _train(self) -> None:
        """
        Train, or re-train, the US College RS.
        """
        df = self.dataset.copy()
        labels = df.pop('name')
        self.model.fit(df.values)
    
    
args = dict(
    choices = ['Georgia Institute of Technology', 
               'Massachusetts Institute of Technology',
               'Brandeis University'],
    metrics = {'gre': 650, 'gpa': 3.5, 'rank': 1},
    top = 4
)
rs = USCollegesRS(args)
rs.recommend()

[{'gre': 640,
  'gpa': 3.63,
  'rank': 1,
  'name': 'Georgia Institute of Technology'},
 {'gre': 660,
  'gpa': 4.0,
  'rank': 1,
  'name': 'Massachusetts Institute of Technology'},
 {'gre': 660, 'gpa': 3.94, 'rank': 2, 'name': 'Brandeis University'}]

In [14]:
rs.dataset

,gre,gpa,rank,name
0,700,3.65,2,University of Illinois at Urbana-Champaign
1,700,3.27,2,Washington University in St Louis
2,640,3.35,3,Creighton University
3,600,3.47,2,University of Wisconsin
4,700,4.00,1,Harvard University
...,...,...,...,...
156,580,3.50,2,Colorado State University
157,560,2.71,3,Florida State University
158,620,3.21,4,Texas Tech University
159,300,3.01,3,Oregon Health and Science University
